In [1]:
import pandas as pd

test_path = "/Users/alexandre/Desktop/X/Python for Data Science/Projet Final Churn/test.parquet"
test = pd.read_parquet(test_path)

print(test.shape)
display(test.head())


(4393179, 19)


,status,gender,firstName,level,lastName,userId,ts,auth,page,sessionId,location,itemInSession,userAgent,method,length,song,artist,time,registration
7,200,M,Jonathan,free,Martin,1465194,1538352006000,Logged In,NextSong,22483,"New York-Newark-Jersey City, NY-NJ-PA",29,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",PUT,250.82730,Mockingbird,Eminem,2018-10-01 00:00:06,2018-09-27 17:29:36
54,200,M,Jonathan,free,Martin,1465194,1538352028000,Logged In,Roll Advert,22483,"New York-Newark-Jersey City, NY-NJ-PA",30,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",GET,NaN,None,None,2018-10-01 00:00:28,2018-09-27 17:29:36
477,200,M,Jonathan,free,Martin,1465194,1538352256000,Logged In,NextSong,22483,"New York-Newark-Jersey City, NY-NJ-PA",31,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",PUT,355.78730,Thank You (Precious Memories Album Version),Ray Boltz,2018-10-01 00:04:16,2018-09-27 17:29:36
1170,200,M,Jonathan,free,Martin,1465194,1538352611000,Logged In,NextSong,22483,"New York-Newark-Jersey City, NY-NJ-PA",32,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",PUT,191.68608,Mathletics,Foals,2018-10-01 00:10:11,2018-09-27 17:29:36
1552,200,M,Jonathan,free,Martin,1465194,1538352802000,Logged In,NextSong,22483,"New York-Newark-Jersey City, NY-NJ-PA",33,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",PUT,275.25179,Proceed,The Roots,2018-10-01 00:13:22,2018-09-27 17:29:36


In [2]:
from functools import reduce

# Convert timestamp
test["ts"] = pd.to_datetime(test["ts"], unit="ms")
test["date"] = test["ts"].dt.date

# 1) total events
f_events_test = (
    test.groupby("userId")
        .size()
        .reset_index(name="n_events")
)

# 2) active days
f_days_test = (
    test.groupby("userId")["date"]
        .nunique()
        .reset_index(name="n_active_days")
)

# 3) unique pages
f_pages_test = (
    test.groupby("userId")["page"]
        .nunique()
        .reset_index(name="n_unique_pages")
)

# 4) sessions
f_sessions_test = (
    test.groupby("userId")["sessionId"]
        .nunique()
        .reset_index(name="n_sessions")
)

# merge everything
feature_dfs_test = [f_events_test, f_days_test, f_pages_test, f_sessions_test]

test_df = reduce(
    lambda left, right: pd.merge(left, right, on="userId", how="left"),
    feature_dfs_test
)

display(test_df.head())
print(test_df.shape)


,userId,n_events,n_active_days,n_unique_pages,n_sessions
0,1000655,346,11,11,10
1,1000963,2539,26,17,21
2,1001129,668,10,15,8
3,1001963,718,16,14,16
4,1002283,3837,25,17,24


(2904, 5)


In [3]:
X_test = test_df.drop(columns=["userId"])


In [5]:
import joblib

xgb = joblib.load("xgb_model.pkl")
print("✅ Model loaded!")


✅ Model loaded!


In [6]:
test_proba = xgb.predict_proba(X_test)[:, 1]


In [7]:
submission = pd.DataFrame({
    "id": test_df["userId"],
    "target": test_proba
})

submission.head()


,id,target
0,1000655,0.006182
1,1000963,0.021906
2,1001129,0.087210
3,1001963,0.018891
4,1002283,0.046848


In [19]:
# convertir proba en 0/1
test_pred_label = (test_proba >= 0.5).astype(int)

submission = pd.DataFrame({
    "id": test_df["userId"],
    "target": test_pred_label
})

print(submission["target"].value_counts())
print(submission.dtypes)

submission.to_csv("submission.csv", index=False)
print("✅ submission.csv (0/1 labels) generated")



target
0    2810
1      94
Name: count, dtype: int64
id        object
target     int64
dtype: object
✅ submission.csv (0/1 labels) generated


In [12]:
example = pd.read_csv("example_submission.csv")


In [21]:
# 1) Convertir les proba en labels 0/1 (seuil 0.5)
test_pred_label = (test_proba >= 0.7).astype(int)

# 2) Créer le DataFrame avec les prédictions
predictions_df = pd.DataFrame({
    "id": test_df["userId"].astype(str),
    "target": test_pred_label
})

# 3) S'assurer que les types matchent
example["id"] = example["id"].astype(str)

# 4) Merge en gardant l'ordre de example
submission = example[["id"]].merge(
    predictions_df,
    on="id",
    how="left"
)

# 5) Gérer le cas ultra rare où un id n’est pas trouvé
submission["target"] = submission["target"].fillna(0).astype(int)

display(submission.head())
print(f"Submission shape: {submission.shape}")
print(submission.dtypes)
submission.to_csv("submission1.csv", index=False)
print(submission["target"].value_counts())


,id,target
0,1128274,0
1,1782451,0
2,1611542,0
3,1241663,0
4,1653104,0


Submission shape: (2904, 2)
id        object
target     int64
dtype: object
target
0    2871
1      33
Name: count, dtype: int64
